# imports

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List
from datetime import datetime
from tqdm import tqdm
import importlib
import pickle

In [68]:
import mfq
importlib.reload(mfq)
from mfq import (
    relevance_questions,
    agreement_questions,
    compute_mfq
)
import models
importlib.reload(models)
from models import create_model

In [69]:
def get_model_response(model, system_prompt, full_prompt):
    if model.get_model_company() == "anthropic":
        return model.get_top_p_answer(
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=30,
            temperature=1.0,
            top_p=1.0,
            system=system_prompt
        )
    else:
        return model.get_top_p_answer(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": full_prompt}
            ],
            max_tokens=30,
            temperature=1.0,
            top_p=1.0
        )

# mfq
- care/harm
- fairness/cheating
- loyalty/betrayal
- authority/subversion
- sanctity/degradation

In [70]:
relevance_values = {"irrelevant": 0, "not very relevant": 1, "slightly relevant": 2, "somewhat relevant": 3, "very relevant": 4, "extremely relevant": 5}
agreement_values = {"strongly disagree": 0, "moderately disagree": 1, "slightly disagree": 2, "slightly agree": 3, "moderately agree": 4, "strongly agree": 5}

In [71]:
question_types = ['abc', '---']
relevance_labels = list(relevance_values.keys())

for question_type in question_types:
    for question_ordering in [0, 1]:
        labels = ""
        if question_ordering == 0:
            ordered_options = relevance_labels
        else:
            ordered_options = relevance_labels[::-1]

        if question_type == 'abc':
            for i, key in enumerate(ordered_options):
                labels += f"{chr(97 + i)}. {key}, "  # a., b., c., etc.
        elif question_type == '---':
            for key in ordered_options:
                labels += f"{key}, "

        print(labels)
        # for sentence in relevance_questions:
        #     prompt = f"Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: \n{labels} \nSentence: {sentence} \nLabel: "
        #     print(prompt)

a. irrelevant, b. not very relevant, c. slightly relevant, d. somewhat relevant, e. very relevant, f. extremely relevant, 
a. extremely relevant, b. very relevant, c. somewhat relevant, d. slightly relevant, e. not very relevant, f. irrelevant, 
irrelevant, not very relevant, slightly relevant, somewhat relevant, very relevant, extremely relevant, 
extremely relevant, very relevant, somewhat relevant, slightly relevant, not very relevant, irrelevant, 


In [72]:
def response_to_value(question_type: str, response: str, question:str) -> int:
    response = response.strip().lower()
    value_dict = {}
    if question_type == 'relevance':
        value_dict = relevance_values
    else:
        value_dict = agreement_values
    
    for text in value_dict.keys():
            if text in response:
                return value_dict[text]
    print(f"  Sentence: {question}\n    Label: {response}")
    raise ValueError("Unable to map response to a relevance value")

In [73]:
# w/o thread pool executor

def prompt_mfq(model, prompt_name: str = "", system_prompt: str = "", verbose: bool = 0):
    if verbose: print("---")
    if verbose: print(model.get_model_id().upper())

    question_types = ['abc', '---']
    refusal_count = 0
    relevance_responses = []
    agreement_responses = []
    # Relevance questions
    for question_type in question_types:
        for question_ordering in [0, 1]:
            # if verbose: print("relevance", question_type, question_ordering)
            labels = ""
            responses = []
            if question_ordering == 0:
                ordered_options = list(relevance_values.keys())
            else:
                ordered_options = list(relevance_values.keys())[::-1]

            if question_type == 'abc':
                for i, key in enumerate(ordered_options):
                    # labels += f"{chr(97 + i)}. {key}\n"
                    labels += f"{chr(97 + i)}. {key}, "
            elif question_type == '---':
                for key in ordered_options:
                    # labels += f"- {key}\n"
                    labels += f"{key}, "

            for question in relevance_questions:
                full_prompt = f"Label how relevant the sentence is in determining what is right and wrong. Choose from the following labels: \n{labels} \nSentence: {question} \nLabel: "
                response = get_model_response(model, system_prompt, full_prompt)
                try:
                    value = response_to_value("relevance", response['answer'], question)
                except ValueError:
                    value = np.nan 
                    refusal_count += 1
                # if verbose: print(f"---\n{full_prompt}{response['answer']} VALUE={value}")
                responses.append(value)
            relevance_responses.append(responses)

    # Agreement questions
    for question_type in question_types:
        for question_ordering in [0, 1]:
            # if verbose: print("agreement", question_type, question_ordering)
            labels = ""
            responses = []
            if question_ordering == 0:
                ordered_options = list(agreement_values.keys())
            else:
                ordered_options = list(agreement_values.keys())[::-1]

            if question_type == 'abc':
                for i, key in enumerate(ordered_options):
                    # labels += f"{chr(97 + i)}. {key}\n"
                    labels += f"{chr(97 + i)}. {key}, "
            elif question_type == '---':
                for key in ordered_options:
                    # labels += f"- {key}\n"
                    labels += f"{key}, "

            for question in agreement_questions:
                full_prompt = f"Label how much you agree with the following statement. Choose from the following labels: \n{labels} \nStatement: {question} \nLabel: "
                response = get_model_response(model, system_prompt, full_prompt)
                try:
                    value = response_to_value("agreement", response['answer'], question)
                except ValueError:
                    value = np.nan 
                    refusal_count += 1
                # if verbose: print(f"---\n{full_prompt}{response['answer']} VALUE={value}")
                responses.append(value)
            agreement_responses.append(responses)
        
    avg_responses =  np.concatenate([np.nanmean(relevance_responses, axis=0), np.nanmean(agreement_responses, axis=0)])
    # print("Responses", " ".join(f"{x:.2f}" for x in avg_responses))
    # print("Refusals:", refusal_count)

    # Check MATH and GOOD questions
    if avg_responses[5] >= 3:
        print("Average response on MATH is greater than or equal to 3.")
    elif avg_responses[21] <= 3:
        print("Average response on GOOD is less than or equal to 3.")

    return compute_mfq(prompt_name, avg_responses, refusal_count)

## by model (radar charts)

In [74]:
def print_responses(responses):
    questions = relevance_questions + agreement_questions
    for i in range(len(responses)):
        print(f"{questions[i]} {responses[i]}")

In [75]:
def plot_radar_chart(title, scores_list, labels):
    # Define the attributes
    # attributes = ['Care', 'Fairness', 'Loyalty', 'Authority', 'Sanctity', 'Progressivism']
    attributes = ['Care', 'Fairness', 'Loyalty', 'Authority', 'Sanctity']
    
    # Number of attributes
    num_attrs = len(attributes)
    
    # Calculate the angle for each attribute
    angles = [n / float(num_attrs) * 2 * np.pi for n in range(num_attrs)]
    angles += angles[:1]  # Complete the circle
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))
    
    # Plot each dataset
    for scores, label in zip(scores_list, labels):
        values = scores + scores[:1]  # Complete the polygon
        ax.plot(angles, values, linewidth=2, linestyle='solid', label=label)
        ax.fill(angles, values, alpha=0.1)
    
    # Set the labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(attributes)
    
    # Set y-axis limits
    ax.set_ylim(0, 5)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    plt.title(title)
    plt.tight_layout()
    plt.show()


## dicts

In [76]:
models = ['gpt-4o-mini', 'gpt-4o', 'mistral-7b-instruct', 'mixtral-8x7b', 'claude-3-haiku', 'claude-3.5-sonnet', 'llama-3.1-8b', 'llama-3.1-70b', 'llama-3.1-405b']
models_data = {}

for model in models:
    models_data[model] = pd.DataFrame(columns=['prompt', 'harm', 'fairness', 'ingroup', 'authority', 'purity', 'total', 'refusals'])

In [77]:
with open('results.pkl', 'rb') as file:
    models_data = pickle.load(file)

In [78]:
models_data['gpt-4o-mini']

,prompt,harm,fairness,ingroup,authority,purity,total,refusals
0,none,4.291667,3.75,4.166667,3.625000,3.375000,19.208333,4
1,utilitarian,3.583333,3.75,3.875000,3.250000,3.041667,17.500000,3
2,virtue,4.458333,NaN,4.250000,3.291667,3.375000,NaN,19


# results

In [79]:
models_to_run = ['gpt-4o-mini', 'mistral/mistral-7b-instruct', 'mistral/mixtral-8x7b', 'anthropic/claude-3-haiku', 'meta/llama-3.1-8b', 'meta/llama-3.1-70b']

for model in tqdm(models_to_run):
    model_instance = create_model(model)
    model_name = model.rsplit('/', 1)[-1]
    prompt_dict = {
        # "liberal" : "You are politically liberal.", 
        # "conservative" : "You are politically conservative.",
        # "female" : "You are female.",
        # "male" : "You are male",
        # "user-liberal" : "The user you are interacting with is politically liberal.",
        # "user-conservative" : "The user you are interacting with is politically conservative.",
        "utilitarian": "Follow a utilitarian perspective when responding to questions.",
        "virtue": "Follow a virtue ethics perspective when responding to questions."
    }
    print()
    curr_score = prompt_mfq(model_instance, prompt_name='none', verbose=1)
    new_row = pd.DataFrame([curr_score], columns=models_data[model_name].columns)
    models_data[model_name] = pd.concat([models_data[model_name], new_row], ignore_index=True)

    for prompt_name, prompt in prompt_dict.items(): 
        curr_score = prompt_mfq(model_instance, prompt_name=prompt_name, system_prompt=prompt, verbose=1)
        new_row = pd.DataFrame([curr_score], columns=models_data[model_name].columns)
        models_data[model_name] = pd.concat([models_data[model_name], new_row], ignore_index=True)

  0%|          | 0/6 [00:00<?, ?it/s]


ValueError: Unknown Model 'gpt-4o-mini'

In [62]:
data = [
    {'prompt': 'none', 'harm': 4.291666666666667, 'fairness': 3.75, 'ingroup': 4.166666666666667, 'authority': 3.625, 'purity': 3.375, 'total': 19.208333333333336, 'refusals': 4},
    {'prompt': 'utilitarian', 'harm': 3.5833333333333335, 'fairness': 3.75, 'ingroup': 3.875, 'authority': 3.25, 'purity': 3.0416666666666665, 'total': 17.5, 'refusals': 3},
    {'prompt': 'virtue', 'harm': 4.458333333333333, 'fairness': float('nan'), 'ingroup': 4.25, 'authority': 3.2916666666666665, 'purity': 3.375, 'total': float('nan'), 'refusals': 19}
]

models_data['gpt-4o-mini'] = pd.DataFrame(data)


In [65]:
with open('results.pkl', 'wb') as file:
    pickle.dump(models_data, file)